In [ ]:
from manim import *

class NepaliInequalitySolver(Scene):
    def construct(self):
        # CONFIGURATION
        # IMPORTANT: Change this to the exact name of your installed Nepali font
        nepali_font = "Noto Sans Devanagari" 
        
        # Colors
        C1 = RED
        C2 = GREEN
        C_SOL = YELLOW

        # DEFINE ZONES
        # Moved further left/up to use corner space
        TL_ORIGIN = UP*3.8 + LEFT*6.9 
        # Moved graph further right to give text more room
        RIGHT_CENTER = RIGHT*3.8 

        # =========================================
        # SECTION 1: TOP-LEFT - Problem Statement
        # =========================================
        
        header_tl = Text("समस्या (Inequalities):", font=nepali_font, color=C_SOL).scale(0.65)
        header_tl.to_corner(UL)
        
        ineq_tex = MathTex(
            r"1) \quad x + y \le 6",
            r"2) \quad 3x + y \le 12",
            r"3) \quad x \ge 0, y \ge 0",
            tex_template=TexTemplateLibrary.default
        ).scale(0.8) # Smaller scale to fit
        
        # Tighter vertical arrangement
        ineq_tex.arrange(DOWN, aligned_edge=LEFT, buff=0.15)
        ineq_tex.next_to(header_tl, DOWN, aligned_edge=LEFT, buff=0.2)
        ineq_tex[0].set_color(C1)
        ineq_tex[1].set_color(C2)

        self.play(Write(header_tl), Write(ineq_tex))


        # =========================================
        # SECTION 2: RIGHT SIDE - Setup Graph
        # =========================================
        
        axes = Axes(
            x_range=[-1, 8, 1],
            y_range=[-1, 14, 2],
            x_length=6.0, 
            y_length=6.0,
            axis_config={"color": WHITE, "include_numbers": True, "font_size": 18}, # Smaller numbers
            tips=False
        ).move_to(RIGHT_CENTER)
        
        labels = axes.get_axis_labels(x_label="x", y_label="y").scale(0.7)
        
        self.play(Create(axes), Write(labels))


        # =========================================
        # SECTION 3: LEFT COLUMN - Detailed Steps
        # =========================================
        
        header_bl = Text("हल गर्ने चरणहरू (Steps):", font=nepali_font, color=BLUE).scale(0.55)
        # Positioned closer to inequalities to save space
        header_bl.next_to(ineq_tex, DOWN, aligned_edge=LEFT, buff=0.5)
        self.play(Write(header_bl))
        
        current_calc_pos = header_bl.get_bottom() + DOWN*0.2

        # --- Helper function for compact steps ---
        def process_line_step(equation_tex, line_color, point1_coords, point2_coords, step_num, arrow_direction):
            nonlocal current_calc_pos
            
            # 1. Step Header (Smaller font)
            step_txt = Text(f"चरण {step_num}: रेखाको लागि बिन्दुहरू", font=nepali_font).scale(0.45)
            step_txt.move_to([TL_ORIGIN[0], current_calc_pos[1], 0], aligned_edge=LEFT)
            
            # Equation copy (Smaller)
            eq_copy = equation_tex.copy().scale(0.7).next_to(step_txt, RIGHT, buff=0.3)
            
            self.play(Write(step_txt), FadeIn(eq_copy))
            
            # Move down just a little bit
            current_calc_pos = step_txt.get_bottom() + DOWN*0.15

            # 2. Calculation Text - Condensed to use less vertical space
            x1, y1 = point1_coords
            x2, y2 = point2_coords

            # Line 1: Calc 1
            calc1_txt = Text(f"१) x={x1} → y={y1}", font=nepali_font, font_size=18, color=GRAY_B)
            calc1_txt.move_to([TL_ORIGIN[0], current_calc_pos[1], 0], aligned_edge=LEFT)
            
            point1_txt = Text(f"बिन्दु १: ({x1}, {y1})", font=nepali_font, font_size=18, color=line_color)
            point1_txt.next_to(calc1_txt, RIGHT, buff=0.3)
            
            # Line 2: Calc 2 (Placed immediately below)
            calc2_txt = Text(f"२) y={y2} → x={x2}", font=nepali_font, font_size=18, color=GRAY_B)
            calc2_txt.next_to(calc1_txt, DOWN, aligned_edge=LEFT, buff=0.1)
            
            point2_txt = Text(f"बिन्दु २: ({x2}, {y2})", font=nepali_font, font_size=18, color=line_color)
            point2_txt.next_to(calc2_txt, RIGHT, buff=0.3)

            # 3. Animate Dots
            self.play(Write(calc1_txt), Write(point1_txt))
            dot1 = Dot(axes.c2p(x1, y1), color=line_color).scale(1.2)
            label1 = MathTex(f"({x1},{y1})", color=line_color).scale(0.5).next_to(dot1, UR, buff=0.05)
            self.play(Indicate(dot1), Write(label1))
            
            self.play(Write(calc2_txt), Write(point2_txt))
            dot2 = Dot(axes.c2p(x2, y2), color=line_color).scale(1.2)
            label_pos = UR if x2 == 0 else UR + UP*0.2 + LEFT*0.5
            label2 = MathTex(f"({x2},{y2})", color=line_color).scale(0.5).next_to(dot2, label_pos, buff=0.05)
            self.play(Indicate(dot2), Write(label2))

            # 4. Draw Extended Line
            extended_line = Line(
                start=axes.c2p(x1 - (x2-x1)*0.2, y1 - (y2-y1)*0.2),
                end=axes.c2p(x2 + (x2-x1)*0.2, y2 + (y2-y1)*0.2),
                color=line_color
            )
            self.play(Create(extended_line))
            
            # 5. Add Region Arrows
            mid_point = axes.c2p((x1+x2)/2, (y1+y2)/2)
            arrow = Arrow(
                start=mid_point, 
                end=mid_point + arrow_direction * 0.8, 
                color=line_color, 
                buff=0,
                stroke_width=3,
                tip_length=0.2
            )
            
            self.play(GrowArrow(arrow))
            
            # Cleanup for next step: Use less buffer to keep it tight
            current_calc_pos = calc2_txt.get_bottom() + DOWN*0.4
            return VGroup(step_txt, eq_copy, calc1_txt, point1_txt, calc2_txt, point2_txt, dot1, label1, dot2, label2, arrow)


        # --- Processing Line 1 (Red) ---
        line1_stuff = process_line_step(
            ineq_tex[0], C1, (0, 6), (6, 0), "१ (रातो)", DL
        )
        self.wait(1)

        # --- Processing Line 2 (Green) ---
        line2_stuff = process_line_step(
            ineq_tex[1], C2, (0, 12), (4, 0), "२ (हरियो)", DL
        )
        self.wait(1)
        
        # --- Processing Axes Constraints ---
        axis_arrow_x = Arrow(start=axes.c2p(0, 0.5), end=axes.c2p(1, 0.5), color=WHITE, buff=0, stroke_width=2)
        axis_arrow_y = Arrow(start=axes.c2p(0.5, 0), end=axes.c2p(0.5, 1), color=WHITE, buff=0, stroke_width=2)
        
        self.play(GrowArrow(axis_arrow_x), GrowArrow(axis_arrow_y))
        self.wait(1)

        # --- Cleanup ---
        self.play(
            FadeOut(line1_stuff),
            FadeOut(line2_stuff),
            FadeOut(header_bl),
            FadeOut(axis_arrow_x),
            FadeOut(axis_arrow_y)
        )

        # =========================================
        # SECTION 4: Final Solution
        # =========================================

        sol_header = Text("अन्तिम समाधान (Final Solution):", font=nepali_font, color=C_SOL).scale(0.55)
        sol_header.next_to(ineq_tex, DOWN, aligned_edge=LEFT, buff=0.5)
        
        sol_text1 = Text("सबै तीरहरूको साझा क्षेत्र (Common Region)।", font=nepali_font, font_size=18).next_to(sol_header, DOWN, aligned_edge=LEFT, buff=0.2)
        
        self.play(Write(sol_header), Write(sol_text1))

        # Vertices: (0,0), (4,0), (3,3), (0,6)
        vertices = [
            axes.c2p(0, 0),
            axes.c2p(4, 0),
            axes.c2p(3, 3),
            axes.c2p(0, 6)
        ]
        
        solution_region = Polygon(*vertices, color=C_SOL, fill_color=C_SOL, fill_opacity=0.4, stroke_width=2)
        inter_dot = Dot(axes.c2p(3,3), color=C_SOL)
        inter_label = MathTex("(3,3)", color=C_SOL).scale(0.6).next_to(inter_dot, UR, buff=0.1)

        self.play(
            GrowFromCenter(solution_region),
            FadeIn(inter_dot),
            Write(inter_label)
        )

        final_lbl = Text("समाधान\nक्षेत्र", font=nepali_font, font_size=24, color=WHITE).move_to(axes.c2p(1.5, 2))
        self.play(Write(final_lbl))
        
        self.wait(3)

%manim -qk -v warning NepaliInequalitySolver